# Imports

In [15]:
import os
import pandas as pd
import psycopg2

from dotenv import load_dotenv
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup

In [16]:
dotenv_path = ("/Users/maukanmir/Documents/Machine-Learning/Web-Scraping-Code/Player-Salaries/dot.env")
load_dotenv(dotenv_path)

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
TABLE_NAME = "player_salaries"

# Functions

In [17]:
def extract_player_info(html_content, year):
    soup = BeautifulSoup(html_content, 'html.parser')
    player_data = []
    players = soup.find_all('li', class_='list-group-item')

    for player in players:
        data = {}

        name_div = player.find('div', class_='link')
        salary_span = player.find('span', class_='medium')
        team_position_small = player.find('small')
        
        if name_div:
            
            data['player'] = name_div.text.strip()

            block = team_position_small.text.strip().split(",")
            team = block[0]
            pos = block[1]
            data['team'] = team
            data["pos"] = pos
            salary = salary_span.text.strip().replace("$", "").replace(",", "")
            data['salary'] = int(salary)
            player_data.append(data)
    
    df = pd.DataFrame(player_data)
    df["season"] = int(year)
    return df


# Grabbing Player Salaries

In [19]:
years = [str(year) for year in range(2011, 2025)]
all_data = pd.DataFrame()

for year in years:
  url = f"https://www.spotrac.com/nba/rankings/player/_/year/{year}/sort/cash_total"
  response = requests.get(url)
  df = extract_player_info(response.text, year)
  all_data = pd.concat([all_data, df])

In [30]:
all_data['id'] = range(1, len(all_data) + 1)

# Save To Posgres

In [34]:
try:
  with psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    ) as conn:
      with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:
        cur.execute(f"DROP TABLE IF EXISTS {TABLE_NAME};")

        cur.execute(f"""
            CREATE TABLE {TABLE_NAME} (
                id INTEGER PRIMARY KEY,
                player VARCHAR(255),
                team VARCHAR(12),
                pos  VARCHAR(20),
                salary INTEGER,
                season INTEGER
            );
        """)
        
        insert_query = f'INSERT INTO {TABLE_NAME} (id, player, team, pos, salary, season) VALUES (%s, %s, %s, %s, %s, %s)'
        rows_to_insert = [(row['id'], row['player'], row['team'], row['pos'], row['salary'], row['season']) for index, row in all_data.iterrows()]
        cur.executemany(insert_query, rows_to_insert)
        
        conn.commit()
        print("Data written to the database.")
except Exception as e:
  print(f"Database Failed to upload Data. The error is: {e}")
        

Data written to the database.
